In [53]:
import pandas as pd

data = pd.read_csv('data1.txt',delimiter='\;', header=None, names = ['comment', 'emotion'])
# In feature engineering we will create 3 features (text_len),(punctuation_perc), and (capital_perc)
import string
#adding feature (text_len)
data['body_len'] = data['comment'].apply(lambda i : len(i)-i.count(" "))
data.emotion.value_counts()



C:\Users\user\AppData\Local\Temp\ipykernel_8100\529907314.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('data1.txt',delimiter='\;', header=None, names = ['comment', 'emotion'])


joy         5362
sadness     4666
love        2692
anger       2159
fear        1937
surprise    1905
Name: emotion, dtype: int64

In [54]:
import re
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub('[^a-zA-Z0-9\s]', '', text)

    # Remove numbers
    text = re.sub('\d+', '', text)

    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()

    return text

# Apply the clean_text function to the content column
data['comment'] = data['comment'].apply(clean_text)

In [55]:

#Tokenization & Removing 'stop words' for training
from nltk.tokenize import word_tokenize
filtered_sentence_arr=[]
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
for example_sent in data['comment']:
    word_tokens = word_tokenize(example_sent)
    # converts the words in word_tokens to lower case and then checks whether
    #they are present in stop_words or not
    filtered_sentence = []
    for w in word_tokens:
        if w.lower() not in stop_words:
            filtered_sentence.append(w.lower())
    filtered_sentence_arr.append(filtered_sentence)
data['comment']=filtered_sentence_arr
data.head()


,comment,emotion,body_len
0,"[didnt, feel, humiliated]",sadness,20
1,"[go, feeling, hopeless, damned, hopeful, aroun...",sadness,88
2,"[im, grabbing, minute, post, feel, greedy, wrong]",anger,39
3,"[ever, feeling, nostalgic, fireplace, know, st...",love,75
4,"[feeling, grouchy]",anger,17


In [56]:
#second (Stemming) & (lemmitization) for training
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()
arr1=[]
arr2=[]
for i in data['comment']:
    arr1=[]
    for j in i:
        a=lemmatizer.lemmatize(j)
        arr1.append(a)
    arr2.append(" ".join(arr1))
data['comment']=arr2

data.head()



,comment,emotion,body_len
0,didnt feel humiliated,sadness,20
1,go feeling hopeless damned hopeful around some...,sadness,88
2,im grabbing minute post feel greedy wrong,anger,39
3,ever feeling nostalgic fireplace know still pr...,love,75
4,feeling grouchy,anger,17


In [57]:
# Extracting TF-IDF parameters
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, analyzer='word',ngram_range=(1,3))
xx = tfidf.fit_transform(data['comment'])

#Training & Testing Encoding
y = data.emotion.replace(to_replace=['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'], value=[0,1,2,3,4,5])
x=pd.concat([data['body_len'],pd.DataFrame(xx)],axis=1)


In [58]:
# Extracting TF-IDF parameters
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xx, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)
x_train


<16848x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 130200 stored elements in Compressed Sparse Row format>

In [59]:
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import KFold, cross_val_score
#rf=RandomForestClassifier(n_jobs=-1)
#k_fold = KFold(n_splits=5)
#cross_val_score(rf,x,data['comment'],cv=k_fold, scoring='accuracy',n_jobs=-1)

In [60]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
acc_naive=accuracy_score(y_pred, y_test)
print('naive bayes tfidf accuracy %s' % accuracy_score(y_pred, y_test))

naive bayes tfidf accuracy 0.6705819540843566


In [61]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
acc_random=accuracy_score(y_pred, y_test)
print('random forest tfidf accuracy %s' % accuracy_score(y_pred, y_test))

random forest tfidf accuracy 0.7293112653497064


In [62]:
x_train, x_test, y_train, y_test = train_test_split(data['comment'], y, stratify=y, random_state=42, test_size=0.1, shuffle=True)
# Extracting Count Vectors Parameters
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='word')
count_vect.fit(data['comment'])
X_train_count =  count_vect.transform(x_train)
X_val_count =  count_vect.transform(x_test)

In [63]:
from sklearn.linear_model import SGDClassifier
lsvm = SGDClassifier(alpha=0.001, random_state=5, max_iter=15, tol=None)
lsvm.fit(X_train_count, y_train)
y_pred = lsvm.predict(X_val_count)
acc_lsvm=accuracy_score(y_pred, y_test)
print('lsvm using count vectors accuracy %s' % x_acc)

lsvm using count vectors accuracy 0.9113721302722905


In [64]:
print('lsvm using count vectors accuracy %s' % x_acc)
arr_name={
0:'sadness',
1:'anger',
2:'love',
3:'surprise',
4:'fear',
5:'joy',
}

lsvm using count vectors accuracy 0.9113721302722905


In [65]:
#example_sent='im grabbing a minute to post i feel greedy wrong'
example_sent=input("enter end to end or enter example  :")
def word_test(example_sent):
    #Tokenization & Removing 'stop words' for training
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = []
    for w in word_tokens:
        if w.lower() not in stop_words:
            filtered_sentence.append(lemmatizer.lemmatize(w.lower()))
            #a=lemmatizer.lemmatize(j)
    filtered_sentence=" ".join(filtered_sentence)
    filtered_sentence=[filtered_sentence]
    example_sent_count =  count_vect.transform(filtered_sentence)
    y_pred = lsvm.predict(example_sent_count)
    
    x= arr_name[y_pred[0]]
    print(x)
    return x
y_pred1=''
y_pred1=word_test(example_sent)
print(y_pred1)

enter end to end or enter example  :
joy
joy


In [66]:
a="""i didnt feel humiliated;sadness
i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake;sadness
im grabbing a minute to post i feel greedy wrong;anger
i am ever feeling nostalgic about the fireplace i will know that it is still on the property;love
i am feeling grouchy;anger
ive been feeling a little burdened lately wasnt sure why that was;sadness
ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny;surprise
i feel as confused about life as a teenager or as jaded as a year old man;fear
i have been with petronas for years i feel that petronas has performed well and made a huge profit;joy
i feel romantic too;love
i feel like i have to make the suffering i m seeing mean something;sadness
i do feel that running is a divine experience and that i can expect to have some type of spiritual encounter;joy
i think it s the easiest time of year to feel dissatisfied;anger
i feel low energy i m just thirsty;sadness
i have immense sympathy with the general point but as a possible proto writer trying to find time to write in the corners of life and with no sign of an agent let alone a publishing contract this feels a little precious;joy
i do not feel reassured anxiety is on each side;joy
i didnt really feel that embarrassed;sadness
i feel pretty pathetic most of the time;sadness
i started feeling sentimental about dolls i had as a child and so began a collection of vintage barbie dolls from the sixties;sadness

"""

In [85]:
from  tkinter import *
from  tkinter.ttk import Combobox
#size
w=500
h=400
#pos
x=100
y=100
###################################################
gui=Tk()
model_name=["lsvm","naive bayes","random forest"]
name_model=StringVar()
com=Combobox(gui,value=model_name,state='readonly',textvariable=name_model)
###################################################show out
def def_train():
    example_sent=ent_k.get()
    acc=word_test(example_sent)
    lab_acc_v=Label(gui,text=acc,width=15,height=5,font=('Times',12))
    lab_acc_v.grid(row=4,column=0,columnspan=2,padx=5)
    #test.grid(row=3,column=1,columnspan=2 ,padx=10, pady=40)
###################################################
###################################################
###################################################show out
def def_train_1():
    a=name_model.get()
    if a=='lsvm':
        acc_value=acc_lsvm
    elif a=='naive bayes':
        acc_value=acc_naive
    elif a=='random forest':
        acc_value=acc_random
    else:
        acc_value=100
    modelacc=Label(gui,text=acc_value,font=('Times',12))
    modelacc.grid(row=1,column=2,columnspan=2,pady=20)
    #test.grid(row=3,column=1,columnspan=2 ,padx=10, pady=40)
###################################################
###################################################
##########################################
lab_w=Label(gui,text="Emotion System Project",fg='red',width=18,height=5,font=('Times',15))
lab_modelacc=Label(gui,text="model acc",font=('Times',12))
acc_value=x_acc
modelacc=Label(gui,text=acc_value,font=('Times',12))
lab_enter=Label(gui,text="enter the text",font=('Times',12))
#################################
ent_k=Entry(gui)
##############
test=Button(gui,text='Test the model',command=def_train,font=('Times',12))
show_acc=Button(gui,text='Show the acc',command=def_train_1,font=('Times',12))
###################################
lab_w.grid(row=0,column=0,columnspan=4, padx=150, pady=0)
lab_modelacc.grid(row=1,column=0)
#modelacc.grid(row=1,column=3,columnspan=2,pady=20)
lab_enter.grid(row=2,column=0)
ent_k.grid(row=2,column=2,columnspan=2)
test.grid(row=3,column=0,columnspan=2 ,padx=10, pady=40)
show_acc.grid(row=3,column=2,columnspan=2 ,padx=10, pady=40)
com.grid(row=4,column=2,columnspan=2,padx=5)
###################################
gui.title("NLP PROJECT")#title name
gui.geometry(f"{w}x{h}+{x}+{y}")#size(num(w) x num(h))+ pos(num(x)+num(y))

gui.mainloop()

joy
